In [1]:
import datetime
import configparser
import json
import requests
from bs4 import BeautifulSoup

In [2]:
config = configparser.ConfigParser()
config.read('.password', 'UTF-8')
 
username = config.get("settings", 'username')
password = config.get("settings", 'password')

In [3]:
session = requests.Session()

# くらしtepcoからでんき家計簿にログイン
param = {
        'ACCOUNTUID': username,
        'PASSWORD': password,
        'HIDEURL': '/ls/pf/ja/pco/mypage/redirect-sso.page?sitekbn=kakeibo',
        'LOGIN': 'EUAS_LOGIN',
        }

header = {
        'Referer': 'https://www.kakeibo.tepco.co.jp/dk/com/menu/'
        }

login = session.post(
        'https://www.kurashi.tepco.co.jp/kpf-login', data=param, headers=header)
login.encoding = login.apparent_encoding

In [4]:
# 月毎データページへの移動リンクで発火するjs関数から、必要な情報を抜き出す
html = BeautifulSoup(login.text, "html.parser")
link_js = html.find(id="frame3").find(class_="box01 box firstBox").find("a")["onclick"]
#print(link_js)
key_officeCd = link_js.split("'")[3]
key_visitNum = link_js.split("'")[5]
key_display = link_js.split("'")[7]
post_to = link_js.split("'")[17]

#print(key_officeCd)
print("***my officeCd***")
#print(key_visitNum)
print("***my visitNum***")
print(key_display)
print(post_to)

***my officeCd***
***my visitNum***
amount
/dk/com/menu/goElectricUsageAmount


In [5]:
# 月毎データの表示ページ に移動
param = {
        'key.officeCd': key_officeCd,
        'key.visitNum': key_visitNum,
        'key.display': key_display
}

header = {
        'Referer':login.url
        }

data_page = session.post(
        'https://www.kakeibo.tepco.co.jp'+post_to, data=param, headers=header)
data_page.encoding = data_page.apparent_encoding

In [6]:
# 30分毎データページへの移動リンクで発火するjs関数から、必要な情報を抜き出す
html = BeautifulSoup(data_page.text, "html.parser")
link_js = html.find(id="motion_area1").find("a")["onclick"]
#print(link_js)

key_officeCd = link_js.split("'")[3]
key_visitNum = link_js.split("'")[5]
post_to = link_js.split("'")[13]

print("***my officeCd***")
#print(key_officeCd)
print("***my visitNum***")
#print(key_visitNum)
print(post_to)

***my officeCd***
***my visitNum***
/dk/syo/electricUsageAmount/goElectricUsage30minGraph


In [7]:
# 30分毎データの表示ページ に移動
param = {
        'officeVisitCd': "%s/%s"%(key_officeCd, key_visitNum),
        'key.officeCd': key_officeCd,
        'key.visitNum': key_visitNum
}

header = {
        'Referer':data_page.url
        }

data_page_30min = session.post(
        'https://www.kakeibo.tepco.co.jp'+post_to, data=param, headers=header)
data_page_30min.encoding = data_page_30min.apparent_encoding

In [8]:
# 30分値表示ページから、数値データを抜き出す
def getData(html_text):
    def getCDATA(text):
        ret = ""
        isInCDATAArea = False
        for l in text.split("\n"):
            l = l.strip()
            if (l == "// <![CDATA["):
                isInCDATAArea = True
            elif (l.strip() == "// ]]>"):
                isInCDATAArea = False
        
            if(isInCDATAArea):
                ret += l + "\n"
        
        return ret
    
    d = {}
    function = ""
    for l in getCDATA(html_text).split("\n"):
        
        if(l.startswith("function")):
            function = l.split("function")[1].split("()")[0].strip()
            if(function.startswith("vbar_usage_grp")):
                d[function] = {}
            
        if(function.startswith("vbar")):
            if(l.startswith("var items = ")):
                d[function]["items"] = json.loads(l.split("=")[-1].strip(";"))
    
    html = BeautifulSoup(html_text, "html.parser")
    date_str = html.find(id="motion_area1").find_all("td")[1].contents[0]
    
    date = datetime.datetime.strptime(date_str.split("\u3000")[0], '%a %b %d %H:%M:%S JST %Y')
    
    return (date, d["vbar_usage_grp"]["items"][0][1:-1])

In [9]:
# 前日移動に必要な情報
def getParamsToMovePrev(html_text):
    html = BeautifulSoup(html_text, "html.parser")
    link_js = html.find(id="motion_area1").find("a")["onclick"]
    #print(link_js)

    key_d = link_js.split("'")[3]
    key_date = link_js.split("'")[5]
    post_to = link_js.split("'")[13]

    return(key_d, key_date, post_to)

In [10]:
# 前日の30分データ表示ページ を取得
def getPrev(key_d, key_date, post_to):
    param = {
        'key.d': key_d,
        'key.date': key_date
    }

    header = {
        'Referer':data_page.url
    }

    data_page_30min = session.post(
            'https://www.kakeibo.tepco.co.jp'+post_to, data=param, headers=header)
    data_page_30min.encoding = data_page_30min.apparent_encoding
    
    return data_page_30min.text

In [11]:
#取得可能な30分値データを、全て取得

current_page = data_page_30min.text

with open("./usage30min.csv", "w") as f:

    while True:
        d, date, post_to = getParamsToMovePrev(current_page)
        current_page = getPrev(d, date, post_to)
        data = getData(current_page)
        if(len(data[1]) != 48):
            break
    
        f.write(data[0].strftime('%Y-%m-%d'))
        for usage in data[1]:
            f.write(" ," + str(usage))
        f.write("\n")
        f.flush()
